In [1]:
#크롬 드라이버 자동 업데이트
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()  # Check if the current version of chromedriver exists
                                      # and if it doesn't e0xist, download it automatically,
                                      # then add chromedriver to path
import re
from datetime import datetime, timedelta
import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
import time

In [2]:
# ID, PW 값
id = 'id'
pw = 'pw'

#Site Open (Driver Load)
target = 'https://selc.or.kr/lms/main/MainView.do'
#path = r'C:\chromedriver.exe'

driver = webdriver.Chrome()  #chromedriver_autoinstaller 이미 설치되어 있으니 신경쓸필요 없다.
driver.get(url=target)

#요소 Timeout 설정 (암묵적 대기)
driver.implicitly_wait(time_to_wait=3)

#팝업 조지기 (가끔 전체화면 켜야될때도 있음)
pop_up = driver.find_elements(By.XPATH, "//img[contains(@alt,'창닫기')]")

for element in pop_up:
    element.click()


#로그인 시작

univ_list = driver.find_element(By.ID, 'univ_nm_back')
driver.execute_script("arguments[0].click();", univ_list)
WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//a[contains(@onclick,'광운대학교')]"))).click()

driver.find_element(By.ID, 'loginbox_user_id_back').clear()
driver.find_element(By.ID, 'loginbox_user_id_back').send_keys(id)
driver.find_element(By.ID, 'loginbox_user_password_back').clear()
driver.find_element(By.ID, 'loginbox_user_password_back').send_keys(pw)
driver.find_element(By.ID, 'btn_login_action').click()

#여기 잘 안먹힐 수 있는데 일단 그냥 수동으로 들가자

driver.find_element(By.XPATH, "//img[contains(@alt,'수업홈')]").click()
driver.get('https://selc.or.kr/lms/lms/class/courseSchedule/doListView.do')
print('메인 화면에 진입 성공했습니다!')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//img[contains(@alt,'수업홈')]"}
  (Session info: chrome=101.0.4951.54)
Stacktrace:
Backtrace:
	Ordinal0 [0x010CA8E3+2402531]
	Ordinal0 [0x0105B011+1945617]
	Ordinal0 [0x00F4C748+837448]
	Ordinal0 [0x00F79330+1020720]
	Ordinal0 [0x00F795CB+1021387]
	Ordinal0 [0x00FA63B2+1205170]
	Ordinal0 [0x00F942F4+1131252]
	Ordinal0 [0x00FA46C2+1197762]
	Ordinal0 [0x00F940C6+1130694]
	Ordinal0 [0x00F6E676+976502]
	Ordinal0 [0x00F6F586+980358]
	GetHandleVerifier [0x0133BC42+2511938]
	GetHandleVerifier [0x0132ED6F+2458991]
	GetHandleVerifier [0x011631AA+575914]
	GetHandleVerifier [0x01161FA6+571302]
	Ordinal0 [0x01061DEB+1973739]
	Ordinal0 [0x01066778+1992568]
	Ordinal0 [0x01066865+1992805]
	Ordinal0 [0x0106FA41+2030145]
	BaseThreadInitThunk [0x776FFA29+25]
	RtlGetAppContainerNamedObjectPath [0x778D7A7E+286]
	RtlGetAppContainerNamedObjectPath [0x778D7A4E+238]


In [3]:
def click_first_no_see_btn():
    result = []
    
    need_watches = driver.find_elements(By.CSS_SELECTOR, '.lec_cont')
    for element in need_watches:
        status = element.find_elements(By.CSS_SELECTOR, '.learn_act_box dd strong')
        if status: #(완료가 존재해야 함)
            if status[0].text == '(미완료)':
                print('미완료 입니다. 첫번째 미완료 element를 return 합니다.')
                no_see = element.find_element(By.CSS_SELECTOR, '.btn_lecture_view a')
                return no_see
    print('시청할 영상이 없습니다.')
    return


#영상을 다볼때까지 기다리기
def wait_until_video_end():
    driver.switch_to.frame("popCourseContent") #iframe 이동
    
    print('영상이 다 끝날때까지 대기를 시작합니다...')

    while True:
        current_minutes = int(driver.find_element(By.ID, 'calcul_time').text.split(':')[1])
        target_minutes = driver.find_element(By.ID, 'total_time').text.strip()
        if target_minutes != '/ 분': #로딩이 되야 분이 뜬다. 주의!
            target_minutes = int(re.findall("\d+", target_minutes)[0])

            #영상 재생이 끝났으면
            if current_minutes >= target_minutes:
                print("끝났슴다")
                break
            else:
                pass
            time.sleep(1)


#나가기 버튼 누르기
def click_exit():
    print('영상을 다봤으니 Exit합니다.')
    driver.find_element(By.CSS_SELECTOR, '.btn_func a').click()  #닫기버튼 클릭
    
    WebDriverWait(driver, 5).until(EC.alert_is_present())
    driver.switch_to.alert.accept()
    driver.refresh()


week_btns = driver.find_elements(By.CSS_SELECTOR, '.module_quick li a')
btn_count = len(week_btns)

for i in range(btn_count):
    print(f'{i+1}주차 탐색 시작...')
    week_btns[i].click()
    week_btns = driver.find_elements(By.CSS_SELECTOR, '.module_quick li a') #DOM 구조가 변경되어 다시 버튼들을 가져온다.
    
    #--메인코드 시작--#
    print('시청해야 할 영상을 확인중입니다...')
    while True:
        no_see = click_first_no_see_btn() #첫번째 미완료 버튼을 찾아서 누르기.
        if no_see:
            print('미완료 버튼을 클릭하겠습니다.')
            no_see.click()
            wait_until_video_end() #영상을 다볼때까지 기다리기
            click_exit() #나가기 버튼 누르기
            
            driver.switch_to.default_content() #iframe 초기화
            
            time.sleep(5) #안정을 위해 delay
        else:
            break

1주차 탐색 시작...
시청해야 할 영상을 확인중입니다...
시청할 영상이 없습니다.
2주차 탐색 시작...
시청해야 할 영상을 확인중입니다...
시청할 영상이 없습니다.
3주차 탐색 시작...
시청해야 할 영상을 확인중입니다...
시청할 영상이 없습니다.
4주차 탐색 시작...
시청해야 할 영상을 확인중입니다...
시청할 영상이 없습니다.
5주차 탐색 시작...
시청해야 할 영상을 확인중입니다...
시청할 영상이 없습니다.
6주차 탐색 시작...
시청해야 할 영상을 확인중입니다...
시청할 영상이 없습니다.
7주차 탐색 시작...
시청해야 할 영상을 확인중입니다...
시청할 영상이 없습니다.
8주차 탐색 시작...
시청해야 할 영상을 확인중입니다...
시청할 영상이 없습니다.
9주차 탐색 시작...
시청해야 할 영상을 확인중입니다...
시청할 영상이 없습니다.
10주차 탐색 시작...
시청해야 할 영상을 확인중입니다...
시청할 영상이 없습니다.
11주차 탐색 시작...
시청해야 할 영상을 확인중입니다...
시청할 영상이 없습니다.
12주차 탐색 시작...
시청해야 할 영상을 확인중입니다...
시청할 영상이 없습니다.
13주차 탐색 시작...
시청해야 할 영상을 확인중입니다...
시청할 영상이 없습니다.
14주차 탐색 시작...
시청해야 할 영상을 확인중입니다...
시청할 영상이 없습니다.
15주차 탐색 시작...
시청해야 할 영상을 확인중입니다...
미완료 입니다. 첫번째 미완료 element를 return 합니다.
미완료 버튼을 클릭하겠습니다.
영상이 다 끝날때까지 대기를 시작합니다...
끝났슴다
영상을 다봤으니 Exit합니다.
미완료 입니다. 첫번째 미완료 element를 return 합니다.
미완료 버튼을 클릭하겠습니다.
영상이 다 끝날때까지 대기를 시작합니다...
끝났슴다
영상을 다봤으니 Exit합니다.
미완료 입니다. 첫번째 미완료 element를 return 합니다.
미완료 버튼을 클릭하겠습니다.
영상이 다 끝날때까지 대기를 시작합니다...
끝났

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=101.0.4951.54)
Stacktrace:
Backtrace:
	Ordinal0 [0x010CA8E3+2402531]
	Ordinal0 [0x0105B011+1945617]
	Ordinal0 [0x00F4C748+837448]
	Ordinal0 [0x00F4F154+848212]
	Ordinal0 [0x00F4F012+847890]
	Ordinal0 [0x00F4F2A0+848544]
	Ordinal0 [0x00F7AD09+1027337]
	Ordinal0 [0x00F6FFE6+983014]
	Ordinal0 [0x00F942AC+1131180]
	Ordinal0 [0x00F6FAA4+981668]
	Ordinal0 [0x00F944C4+1131716]
	Ordinal0 [0x00FA46C2+1197762]
	Ordinal0 [0x00F940C6+1130694]
	Ordinal0 [0x00F6E676+976502]
	Ordinal0 [0x00F6F586+980358]
	GetHandleVerifier [0x0133BC42+2511938]
	GetHandleVerifier [0x0132ED6F+2458991]
	GetHandleVerifier [0x011631AA+575914]
	GetHandleVerifier [0x01161FA6+571302]
	Ordinal0 [0x01061DEB+1973739]
	Ordinal0 [0x01066778+1992568]
	Ordinal0 [0x01066865+1992805]
	Ordinal0 [0x0106FA41+2030145]
	BaseThreadInitThunk [0x776FFA29+25]
	RtlGetAppContainerNamedObjectPath [0x778D7A7E+286]